# Fog-RTX Work With Cloud
FogRTX supports a wide range of cloud service providers.

### AWS

In [1]:
! git clone https://github.com/KeplerC/fog_x.git
! cd fog_x && git checkout cloud && pip install .

fatal: destination path 'fog_x' already exists and is not an empty directory.
error: pathspec 'cloud' did not match any file(s) known to git


Install required AWS dependency and configure with your aws credential

In [ ]:
! curl "https://awscli.amazonaws.com/awscli-exe-linux-x86_64.zip" -o "awscliv2.zip"
! unzip awscliv2.zip
! sudo ./aws/install
! aws configure

In [ ]:
# create an AWS bucket named fog-rtx-test-east-1 (anything you want)
!aws s3api create-bucket --bucket fog-rtx-test-east-1 --region us-east-1

In [ ]:
# (Optional) set the AWS s3 bucket to be public
! aws aws s3api put-public-access-block \
    --bucket fog-rtx-test-east-1 \
    --public-access-block-configuration "BlockPublicAcls=false,IgnorePublicAcls=false,BlockPublicPolicy=false,RestrictPublicBuckets=false"

Fog-RTX can load from the existing bucket, and add more to it!

In [8]:
import fog_x

dataset = fog_x.dataset.Dataset(
    name="demo_ds",
    path='s3://fog-rtx-test-east-1',
)

I 2024-04-21 19:19:44,550 dataset.py:91] Path does not exist. (s3://fog-rtx-test-east-1/demo_dss)
I 2024-04-21 19:19:44,553 dataset.py:94] Creating s3 bucket...
I 2024-04-21 19:19:44,946 dataset.py:100] Bucket 'fog-rtx-test-east-1' created in AWS S3.
I 2024-04-21 19:19:45,388 polars_connector.py:144] Prepare to load table demo_dss loaded from s3://fog-rtx-test-east-1/demo_dss.parquet.
W 2024-04-21 19:19:45,768 polars_connector.py:154] Failed to load table demo_dss from s3://fog-rtx-test-east-1/demo_dss.parquet.
E 2024-04-21 19:19:45,769 polars_connector.py:118] Table demo_dss does not exist, available tables are dict_keys([]).


read from exisitng AWS stored dataset

In [3]:
dataset.get_episode_info()

episode_id,Finished,feature_gripper_closedness_action_type,feature_gripper_closedness_action_shape,gripper_closedness_action_count,feature_rotation_delta_type,feature_rotation_delta_shape,rotation_delta_count,feature_terminate_episode_type,feature_terminate_episode_shape,terminate_episode_count,feature_world_vector_type,feature_world_vector_shape,world_vector_count,feature_is_first_type,feature_is_first_shape,is_first_count,feature_is_last_type,feature_is_last_shape,is_last_count,feature_is_terminal_type,feature_is_terminal_shape,is_terminal_count,feature_hand_image_type,feature_hand_image_shape,hand_image_count,feature_image_type,feature_image_shape,image_count,feature_image_with_depth_type,feature_image_with_depth_shape,image_with_depth_count,feature_natural_language_embedding_type,feature_natural_language_embedding_shape,natural_language_embedding_count,feature_natural_language_instruction_type,feature_natural_language_instruction_shape,natural_language_instruction_count,feature_robot_state_type,feature_robot_state_shape,robot_state_count,feature_reward_type,feature_reward_shape,reward_count
i64,bool,str,str,f64,str,str,f64,str,str,f64,str,str,f64,str,str,f64,str,str,f64,str,str,f64,str,str,f64,str,str,f64,str,str,f64,str,str,f64,str,str,f64,str,str,f64,str,str,f64
0,true,"""float32""","""()""",123.0,"""float32""","""(3,)""",123.0,"""float32""","""()""",123.0,"""float32""","""(3,)""",123.0,"""bool""","""()""",123.0,"""bool""","""()""",123.0,"""bool""","""()""",123.0,"""uint8""","""(480, 640, 3)""",123.0,"""uint8""","""(480, 640, 3)""",123.0,"""float32""","""(480, 640, 1)""",123.0,"""float32""","""(512,)""",123.0,"""string""","""()""",123.0,"""float32""","""(15,)""",123.0,"""float32""","""()""",123.0
1,true,"""float32""","""()""",103.0,"""float32""","""(3,)""",103.0,"""float32""","""()""",103.0,"""float32""","""(3,)""",103.0,"""bool""","""()""",103.0,"""bool""","""()""",103.0,"""bool""","""()""",103.0,"""uint8""","""(480, 640, 3)""",103.0,"""uint8""","""(480, 640, 3)""",103.0,"""float32""","""(480, 640, 1)""",103.0,"""float32""","""(512,)""",103.0,"""string""","""()""",103.0,"""float32""","""(15,)""",103.0,"""float32""","""()""",103.0
2,true,"""float32""","""()""",110.0,"""float32""","""(3,)""",110.0,"""float32""","""()""",110.0,"""float32""","""(3,)""",110.0,"""bool""","""()""",110.0,"""bool""","""()""",110.0,"""bool""","""()""",110.0,"""uint8""","""(480, 640, 3)""",110.0,"""uint8""","""(480, 640, 3)""",110.0,"""float32""","""(480, 640, 1)""",110.0,"""float32""","""(512,)""",110.0,"""string""","""()""",110.0,"""float32""","""(15,)""",110.0,"""float32""","""()""",110.0
3,true,"""float32""","""()""",118.0,"""float32""","""(3,)""",118.0,"""float32""","""()""",118.0,"""float32""","""(3,)""",118.0,"""bool""","""()""",118.0,"""bool""","""()""",118.0,"""bool""","""()""",118.0,"""uint8""","""(480, 640, 3)""",118.0,"""uint8""","""(480, 640, 3)""",118.0,"""float32""","""(480, 640, 1)""",118.0,"""float32""","""(512,)""",118.0,"""string""","""()""",118.0,"""float32""","""(15,)""",118.0,"""float32""","""()""",118.0
4,true,"""float32""","""()""",84.0,"""float32""","""(3,)""",84.0,"""float32""","""()""",84.0,"""float32""","""(3,)""",84.0,"""bool""","""()""",84.0,"""bool""","""()""",84.0,"""bool""","""()""",84.0,"""uint8""","""(480, 640, 3)""",84.0,"""uint8""","""(480, 640, 3)""",84.0,"""float32""","""(480, 640, 1)""",84.0,"""float32""","""(512,)""",84.0,"""string""","""()""",84.0,"""float32""","""(15,)""",84.0,"""float32""","""()""",84.0
5,true,"""float32""","""()""",97.0,"""float32""","""(3,)""",97.0,"""float32""","""()""",97.0,"""float32""","""(3,)""",97.0,"""bool""","""()""",97.0,"""bool""","""()""",97.0,"""bool""","""()""",97.0,"""uint8""","""(480, 640, 3)""",97.0,"""uint8""","""(480, 640, 3)""",97.0,"""float32""","""(480, 640, 1)""",97.0,"""float32""","""(512,)""",97.0,"""string""","""()""",97.0,"""float32""","""(15,)""",97.0,"""float32""","""()""",97.0


add more to the stored dataset

In [5]:
dataset.load_rtx_episodes(
    name="berkeley_autolab_ur5",
    split="train[4:10]",
)

I 2024-04-21 19:02:52,041 dataset_info.py:610] Load dataset info from gs://gresearch/robotics/berkeley_autolab_ur5/0.1.0
I 2024-04-21 19:02:52,774 reader.py:261] Creating a tf.data.Dataset reading 4 files located in folders: gs://gresearch/robotics/berkeley_autolab_ur5/0.1.0.
I 2024-04-21 19:02:52,902 logging_logger.py:49] Constructing tf.data.Dataset berkeley_autolab_ur5 for split train[4:10], from gs://gresearch/robotics/berkeley_autolab_ur5/0.1.0
I 2024-04-21 19:02:54,578 dataset.py:359] {'steps': <_VariantDataset element_spec={'action': {'gripper_closedness_action': TensorSpec(shape=(), dtype=tf.float32, name=None), 'rotation_delta': TensorSpec(shape=(3,), dtype=tf.float32, name=None), 'terminate_episode': TensorSpec(shape=(), dtype=tf.float32, name=None), 'world_vector': TensorSpec(shape=(3,), dtype=tf.float32, name=None)}, 'is_first': TensorSpec(shape=(), dtype=tf.bool, name=None), 'is_last': TensorSpec(shape=(), dtype=tf.bool, name=None), 'is_terminal': TensorSpec(shape=(), dtyp

The data is automatically uploaded to the cloud!
We can create a separate reader by

In [6]:
dataset2 = fog_x.dataset.Dataset(
    name="demo_ds",
    path='s3://fog-rtx-test-east-1',
)

I 2024-04-21 19:05:24,680 polars_connector.py:144] Prepare to load table demo_ds loaded from s3://fog-rtx-test-east-1/demo_ds.parquet.
I 2024-04-21 19:05:25,837 polars_connector.py:152] Table demo_ds loaded from s3://fog-rtx-test-east-1/demo_ds.parquet.


In [7]:
# metadata
trajectory_metadata = dataset2.get_episode_info()
trajectory_metadata

episode_id,Finished,feature_gripper_closedness_action_type,feature_gripper_closedness_action_shape,gripper_closedness_action_count,feature_rotation_delta_type,feature_rotation_delta_shape,rotation_delta_count,feature_terminate_episode_type,feature_terminate_episode_shape,terminate_episode_count,feature_world_vector_type,feature_world_vector_shape,world_vector_count,feature_is_first_type,feature_is_first_shape,is_first_count,feature_is_last_type,feature_is_last_shape,is_last_count,feature_is_terminal_type,feature_is_terminal_shape,is_terminal_count,feature_hand_image_type,feature_hand_image_shape,hand_image_count,feature_image_type,feature_image_shape,image_count,feature_image_with_depth_type,feature_image_with_depth_shape,image_with_depth_count,feature_natural_language_embedding_type,feature_natural_language_embedding_shape,natural_language_embedding_count,feature_natural_language_instruction_type,feature_natural_language_instruction_shape,natural_language_instruction_count,feature_robot_state_type,feature_robot_state_shape,robot_state_count,feature_reward_type,feature_reward_shape,reward_count
i64,bool,str,str,f64,str,str,f64,str,str,f64,str,str,f64,str,str,f64,str,str,f64,str,str,f64,str,str,f64,str,str,f64,str,str,f64,str,str,f64,str,str,f64,str,str,f64,str,str,f64
0,true,"""float32""","""()""",123.0,"""float32""","""(3,)""",123.0,"""float32""","""()""",123.0,"""float32""","""(3,)""",123.0,"""bool""","""()""",123.0,"""bool""","""()""",123.0,"""bool""","""()""",123.0,"""uint8""","""(480, 640, 3)""",123.0,"""uint8""","""(480, 640, 3)""",123.0,"""float32""","""(480, 640, 1)""",123.0,"""float32""","""(512,)""",123.0,"""string""","""()""",123.0,"""float32""","""(15,)""",123.0,"""float32""","""()""",123.0
1,true,"""float32""","""()""",103.0,"""float32""","""(3,)""",103.0,"""float32""","""()""",103.0,"""float32""","""(3,)""",103.0,"""bool""","""()""",103.0,"""bool""","""()""",103.0,"""bool""","""()""",103.0,"""uint8""","""(480, 640, 3)""",103.0,"""uint8""","""(480, 640, 3)""",103.0,"""float32""","""(480, 640, 1)""",103.0,"""float32""","""(512,)""",103.0,"""string""","""()""",103.0,"""float32""","""(15,)""",103.0,"""float32""","""()""",103.0
2,true,"""float32""","""()""",110.0,"""float32""","""(3,)""",110.0,"""float32""","""()""",110.0,"""float32""","""(3,)""",110.0,"""bool""","""()""",110.0,"""bool""","""()""",110.0,"""bool""","""()""",110.0,"""uint8""","""(480, 640, 3)""",110.0,"""uint8""","""(480, 640, 3)""",110.0,"""float32""","""(480, 640, 1)""",110.0,"""float32""","""(512,)""",110.0,"""string""","""()""",110.0,"""float32""","""(15,)""",110.0,"""float32""","""()""",110.0
3,true,"""float32""","""()""",118.0,"""float32""","""(3,)""",118.0,"""float32""","""()""",118.0,"""float32""","""(3,)""",118.0,"""bool""","""()""",118.0,"""bool""","""()""",118.0,"""bool""","""()""",118.0,"""uint8""","""(480, 640, 3)""",118.0,"""uint8""","""(480, 640, 3)""",118.0,"""float32""","""(480, 640, 1)""",118.0,"""float32""","""(512,)""",118.0,"""string""","""()""",118.0,"""float32""","""(15,)""",118.0,"""float32""","""()""",118.0
4,true,"""float32""","""()""",84.0,"""float32""","""(3,)""",84.0,"""float32""","""()""",84.0,"""float32""","""(3,)""",84.0,"""bool""","""()""",84.0,"""bool""","""()""",84.0,"""bool""","""()""",84.0,"""uint8""","""(480, 640, 3)""",84.0,"""uint8""","""(480, 640, 3)""",84.0,"""float32""","""(480, 640, 1)""",84.0,"""float32""","""(512,)""",84.0,"""string""","""()""",84.0,"""float32""","""(15,)""",84.0,"""float32""","""()""",84.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
7,true,"""float32""","""()""",103.0,"""float32""","""(3,)""",103.0,"""float32""","""()""",103.0,"""float32""","""(3,)""",103.0,"""bool""","""()""",103.0,"""bool""","""()""",103.0,"""bool""","""()""",103.0,"""uint8""","""(480, 640, 3)""",103.0,"""uint8""","""(480, 640, 3)""",103.0,"""float32""","""(480, 640, 1)""",103.0,"""float32""","""(512,)""",103.0,"""string""","""()""",103.0,"""float32""","""(15,)""",103.0,"""float

# Google Cloud Platform

Register google cloud credentials

Alternative in non-colab environment, run following command instead:
```
gcloud auth application-default login   --quiet --no-launch-browser
```


In [ ]:
from google.colab import auth
PROJECT_ID = "canvas-rampart-342500"
auth.authenticate_user(project_id=PROJECT_ID)

In [ ]:
! gcloud storage buckets create gs://fog_rtx_test  --location=us-east1

In [ ]:
dataset = fog_x.dataset.Dataset(
    name="demo_ds",
    path='gs://fog_rtx_test/',
)

### Known issues

1. `export` as rlds format to the cloud directly does not work yet for S3 (known issue for tensorflow Gfile)
2. (will fix) automatically check the existence